# Simulation Benchmarks for FMI.jl
by Tobias Thummerer

Last update: 07.11.2023

🚧 This benchmark is under development, so do not give too much weight to the current results. 🚧

The purpose of this benchmark is to keep track of changes, that influence the simulation performance over different releases. Idealy, simulation time and memory usage should decrease with every new release, but new features and corrections could lead to slightly decreased performances.

Simulation of a vehicle longitudinal dynamics model with six states and time events triggered with 100 Hz. Additional state events, but less than time events. See solver log for details.

## License

In [1]:
# Copyright (c) 2023 Tobias Thummerer, Lars Mikelsons
# Licensed under the MIT license. 
# See LICENSE (https://github.com/thummeto/FMI.jl/blob/main/LICENSE) file in the project root for details.

In [2]:
# we use distributed to check out different FMI.jl versions over different processes
using Distributed 
using Plots
 
# versions to check
versions = ("0.11.0", "0.12.0", "0.12.4", "PATHC:\\Users\\thummeto\\Documents\\FMI.jl") # , "0.13.0")
numVersions = length(versions)

# one worker per version
workers = addprocs(numVersions);

In [3]:
# include benchmarking code
include(joinpath(@__DIR__, "benchmark_helper.jl"));

In [4]:
# install a specific version of FMI.jl on every worker process 
@suppress begin
    futures = Vector{Any}(undef, numVersions);
    for i in 1:numVersions
        futures[i] = @spawnat workers[i] setupVersion(versions[i]);
    end
    fetch.(futures);
end

4-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing

In [5]:
@everywhere workers using FMI, FMIZoo
@everywhere workers data = FMIZoo.VLDM(:train)
@everywhere workers fmu = fmiLoad("VLDM", "Dymola", "2020x"; type=:ME)
@everywhere workers using FMI.DifferentialEquations

@everywhere workers realBuffer = zeros(fmi2Real, 2)
@everywhere workers realVRs = vcat(fmu.modelDescription.stateValueReferences, fmu.modelDescription.derivativeValueReferences)

@everywhere workers x0=zeros(fmi2Real, 6)
@everywhere workers grad = (p, x0, fmu, data, solver) -> ReverseDiff.gradient((_p) -> loss(_p, x0, fmu, data, solver), p)

┌ Warning: The call to compilecache failed to create a usable precompiled cache file for FMI [14a09403-18e3-468f-ad8a-74f8dda2d9ac]
│   exception = ErrorException("Required dependency BoundaryValueDiffEq [764a87c0-6b3e-53db-9096-fe964310641d] failed to load from a cache file.")
└ @ Base loading.jl:1818


      From worker 4:	┌ Warning: Module DiffEqBase with build ID fafbfcfd-52ef-2d66-0000-0fdb79eec1e1 is missing from the cache.
      From worker 4:	│ This may mean DiffEqBase [2b5f629d-d688-5b77-993f-72d75c75574e] does not support precompilation but is imported by a module that does.
      From worker 4:	└ @ Base loading.jl:1793
      From worker 3:	┌ Warning: Module DiffEqBase with build ID fafbfcfd-52ef-2d66-0000-0fdb79eec1e1 is missing from the cache.
      From worker 3:	│ This may mean DiffEqBase [2b5f629d-d688-5b77-993f-72d75c75574e] does not support precompilation but is imported by a module that does.
      From worker 3:	└ @ Base loading.jl:1793
      From worker 2:	┌ Warning: Module LogExpFunctionsChainRulesCoreExt with build ID fafbfcfd-2610-10de-0000-a8c6cbda52ed is missing from the cache.
      From worker 2:	│ This may mean LogExpFunctionsChainRulesCoreExt [1bf5f11d-9a0a-5d25-85d0-d1d9a28a239c] does not support precompilation but is imported by a module that does.
      

      From worker 2:	┌ Error: Error during loading of extension LogExpFunctionsChainRulesCoreExt of LogExpFunctions, use `Base.retry_load_extensions()` to retry.
      From worker 2:	│   exception =
      From worker 2:	│    1-element ExceptionStack:
      From worker 2:	│    Declaring __precompile__(false) is not allowed in files that are being precompiled.
      From worker 2:	│    Stacktrace:
      From worker 2:	│      [1] _require(pkg::Base.PkgId, env::Nothing)
      From worker 2:	│        @ Base .\loading.jl:1797
      From worker 2:	│      [2] _require_prelocked(uuidkey::Base.PkgId, env::Nothing)
      From worker 2:	│        @ Base .\loading.jl:1660
      From worker 2:	│      [3] _require_prelocked(uuidkey::Base.PkgId)
      From worker 2:	│        @ Base .\loading.jl:1658
      From worker 2:	│      [4] run_extension_callbacks(extid::Base.ExtensionId)
      From worker 2:	│        @ Base .\loading.jl:1255
      From worker 2:	│      [5] run_extension_callbacks(pkgid::Base.Pk

      From worker 2:	┌ Error: Error during loading of extension SpecialFunctionsChainRulesCoreExt of SpecialFunctions, use `Base.retry_load_extensions()` to retry.
      From worker 2:	│   exception =
      From worker 2:	│    1-element ExceptionStack:


      From worker 2:	│    Declaring __precompile__(false) is not allowed in files that are being precompiled.
      From worker 2:	│    Stacktrace:
      From worker 2:	│      [1] _require(pkg::Base.PkgId, env::Nothing)
      From worker 2:	│        @ Base .\loading.jl:1797
      From worker 2:	│      [2] _require_prelocked(uuidkey::Base.PkgId, env::Nothing)
      From worker 2:	│        @ Base .\loading.jl:1660
      From worker 2:	│      [3] _require_prelocked(uuidkey::Base.PkgId)
      From worker 2:	│        @ Base .\loading.jl:1658
      From worker 2:	│      [4] run_extension_callbacks(extid::Base.ExtensionId)
      From worker 2:	│        @ Base .\loading.jl:1255
      From worker 2:	│      [5] run_extension_callbacks(pkgid::Base.PkgId)
      From worker 2:	│        @ Base .\loading.jl:1290
      From worker 2:	│      [6] run_package_callbacks(modkey::Base.PkgId)
      From worker 2:	│        @ Base .\loading.jl:1124
      From worker 2:	│      [7] _require_prelocked(uuidkey::Ba

...


      From worker 4:	│ This may mean Zygote [e88e6eb3-aa80-5325-afca-941959d7151f] does not support precompilation but is imported by a module that does.


      From worker 4:	└ @ Base loading.jl:1793


      From worker 3:	┌ Warning: Module Zygote with build ID ffffffff-ffff-ffff-0000-052033fd61d5 is missing from the cache.
      From worker 3:	│ This may mean Zygote [e88e6eb3-aa80-5325-afca-941959d7151f] does not support precompilation but is imported by a module that does.
      From worker 3:	└ @ Base loading.jl:1793

      From worker 4:	┌ Warning: Module Zygote with build ID ffffffff-ffff-ffff-0000-05203b50ca85 is missing from the cache.
      From worker 4:	│ This may mean Zygote [e88e6eb3-aa80-5325-afca-941959d7151f] does not support precompilation but is imported by a module that does.
      From worker 4:	└ @ Base loading.jl:1793


      From worker 2:	┌ Warning: Module OrdinaryDiffEq with build ID ffffffff-ffff-ffff-0000-04b8001a57e9 is missing from the cache.
      From worker 2:	│ This may mean OrdinaryDiffEq [1dea7af3-3e70-54e6-95c3-0bf5283fa5ed] does not support precompilation but is imported by a module that does.
      From worker 2:	└ @ Base loading.jl:1793

In [ ]:
# check the installed versions per process
for w in workers 
    @info "Worker #$(w)"
    sleep(1.0)
    f = @spawnat w Pkg.status("FMI")
    fetch(f) # Pkg.status is non-blocking, so we need to sleep...
    sleep(2.0)
end

In [ ]:
# only load FMISensitivity.jl on the last worker, because there is no support in FMI.jl versions < 0.13.0
@everywhere workers[end] using FMISensitivity

## Benchmark 1a: Explicit Solver

Model: VLDM (6 states, time- and state-events)

Mode: Model-Exchange

Solver: Tsit5 (tolerance=1e-4)

In [ ]:
@everywhere workers solver=Tsit5()
@everywhere workers b=benchmarkSimulation(fmu, data, solver)
min_times, memories, allocs = runBenchmark()
resultPlot(versions, min_times, memories)

## Benchmark 1b: Implicit Solver (using Automatic Differentiation)

Model: VLDM (6 states, time- and state-events)

Mode: Model-Exchange

Solver: Rosenbrock23 (autodiff=true, tolerance=1e-4)

In [ ]:
@everywhere workers solver=Rosenbrock23(autodiff=true)
@everywhere workers b=benchmarkSimulation(fmu, data, solver)
min_times, memories, allocs = runBenchmark()
resultPlot(versions, min_times, memories)

## Benchmark 2: Automatic Solver Determination

Model: VLDM (6 states, time- and state-events)

Mode: Model-Exchange

Solver: auto (by DifferentialEquations.jl)

In [ ]:
@everywhere workers solver=nothing
@everywhere workers b=benchmarkSimulation(fmu, data, solver)
min_times, memories, allocs = runBenchmark()
resultPlot(versions, min_times, memories)

## Benchmark 3: Non-Julia Solver

Model: VLDM (6 states, time- and state-events)

Mode: Model-Exchange

Solver: CVode (tolerance=1e-4)

In [ ]:
@everywhere workers solver = CVODE_BDF()
@everywhere workers b=benchmarkSimulation(fmu, data, solver)
min_times, memories, allocs = runBenchmark()
resultPlot(versions, min_times, memories)